In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp
from pylcp.common import progressBar
import scipy.constants as const
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D 

In [5]:
# SI units
frq_real=603976506.6e6*2*np.pi
gamma_real=61.4e6
kmag_real=frq_real/const.c
muB_real=const.physical_constants["Bohr magneton"][0]
mass_real=const.value('atomic mass constant')*88
alpha_real= 0.4 #in T/m

# Natural units for computation
gamma=1
kmag=1
muB=1
mass=mass_real*gamma_real/const.hbar/kmag_real**2
alpha = alpha_real*muB_real/(gamma_real*kmag_real*const.hbar)
laser_det = 0
det = -2.1*gamma
s = 2
transform = True

# Defining the laser beams. careful: the detuning here seems to be zero. instead of detuning the laser light, the detuning was added to the states by shifting the energy. the result is the same.
laserBeams = pylcp.conventional3DMOTBeams(k=kmag,
    s=s, delta=0., beam_type=pylcp.infinitePlaneWaveBeam
)

# Magnetic field for quadrupole trap
magField = pylcp.quadrupoleMagneticField(alpha)


# Atomic Hamiltonian for 3P2->3D3. No other states are considered as it would drastically increase computational intensity.
H_g, muq_g = pylcp.hamiltonians.singleF(F=2, gF=1.5, muB=muB)
H_e, muq_e = pylcp.hamiltonians.singleF(F=3, gF=1+1/3, muB=muB)
d_q = pylcp.hamiltonians.dqij_two_bare_hyperfine(2, 3)
hamiltonian = pylcp.hamiltonian(H_g, -det*np.eye(7)+H_e, muq_g, muq_e, d_q, mass=mass, muB=muB, gamma=gamma,k=kmag)

# Adding everything into a gouverning equation. The OBE are used to include coherence.
obe = pylcp.obe(laserBeams, magField, hamiltonian,
                transform_into_re_im=transform)

In [ ]:
if hasattr(obe, 'sol'):
    del obe.sol

tmax = 20e5
args = ([0, tmax], )
kwargs = {'t_eval': np.linspace(0, tmax, 20000),
          'random_recoil': True,
          'progress_bar': True, 
          'max_scatter_probability': 0.5,
          'record_force': False,
          'events': []}


# for the starting positions
rscale = np.array([2, 2, 2]) / alpha
roffset = np.array([0.0, 0.0, 0.0])
vscale = np.array([0.1, 0.1, 0.1])
voffset = np.array([0.0, 0.0, 0.0])


# this sets up an atom and calculates its motion
def generate_random_solution(x, tmax=20e5):
    np.random.rand(256 * x)  # de-sync random states
    obe.set_initial_position(rscale * np.random.randn(3) + roffset)
    obe.set_initial_velocity(vscale * np.random.randn(3)*7 + voffset)
    obe.set_initial_rho_equally()
    obe.evolve_motion(*args, **kwargs)
    return obe.sol
 
Natoms = 1 #number of atoms to simulate
sols = []
progress = progressBar()

# choose if you want to simulate everything (took me 5 days) or import the results

# sets up and simulates one by one and adds the solution into a list
for i in range(Natoms):
    sols.append(generate_random_solution(i))
    progress.update((i + 1) / Natoms)
    print('Completed: '+str(i))